* 序列标注应用，以及图模型
* HMM 隐马尔可夫模型 “一模型，二假设，三问题。” 问题一的解决办法：前向算法，反向算法。问题三的解决办法：维特比算法+集束搜索。
* HMM -> MEMM 最大熵马尔可夫模型（优点：打破观测独立假设，缺点：标注偏差）-> CRF（打破HMM的两个假设，以及解决标注偏差问题）
* CRF 条件随机场：



<img src="NLP_github/sequence_labelling.png" width="400" height="400">

<img src="NLP_github/graph_model.png" width="500" height="500">

## 2. 隐马尔可夫模型 HMM - 有向图 - 概率生成模型

概率生成模型与判别模型与机器学习中生成以及判别模型相似。概率生成模型通过联合概率P(X, Y)建模，概率判别模型通过条件概率 P(Y|X)建模。

**“一模型，两假设，三问题。”**

### 2.1 隐马尔可夫模型：

隐马尔可夫是统计模型，它用来描述一个含有隐含未知参数的马尔可夫过程。其难点是从可观察的参数中确定该过程的隐含参数，然后利用这些参数来作进一步的分析。https://www.cnblogs.com/skyme/p/4651331.html

<img src="NLP_github/HMM.png" width="400" height="400">

* 状态序列：$I = i_1, i_2, \dots, i_t $. $\text{Q} = {q_1, q_2, \dots, q_n}$ 状态序列为隐变量。

* 观测序列：$O = o_1, o_2, \dots, o_t $. $\text{V} = {v_1, v_2, \dots, v_m}$，观测序列为观测的结果。

* 模型使用 $\lambda = (\pi, A, B)$ 表示。其中，$\pi$为出始概率分布，A为状态转移矩阵，B为发射矩阵。

* A 状态转移矩阵：$A[a_{ij}] = P(i_{t+1}|i_t)$。状态序列，i+1时刻下的概率仅与i时刻有关。**马尔可夫假设**

* B 发射矩阵：$B[b_{jk}] = P(o_k|i_j)$。观测序列$o_k$对应的概率仅与其对应的状态有关。**观测独立假设**

### 2.2 隐马尔可夫三个基本问题：

>1. 概率问题：给定模型计算观测序列出现的概率 $\text{P}(\text{O}|\lambda)$。**已知隐含状态数量以及转换概率，根据可见状态链，计算此状态的概率。**
2. 机器学习问题：已知观测序列求解模型参数。**已知隐含状态数量以及可见状态链（观测序列），反推转换概率。**
3. **Decoding问题**：给定观测序列，求解最优的状态序列 $argmax P(I|O, \lambda)$。**已知隐含状态数量以及转换概率，根据可见状态链，反推隐含状态链。**
>
><img src="NLP_github/HMM_1.png" width="400" height="400">
>
>### 2.2.1 问题一：给定$\lambda$求$\text{P}(\text{O}|\lambda)$
>
>$$
\text{P}(\text{O}|\lambda) = \sum_\text{I} \text{P}(\text{I}, \text{O}|\lambda) = \sum_\text{I} \text{P}(\text{O}|\text{I}, \lambda) \text{P}(\text{I}|\lambda)
$$
>
>$$
\text{P}(\text{I}|\lambda) = \text{P}(i_t, i_{t-1}, \dots, i, \lambda) = \text{P}(i_t|i_{t-1}, \dots, i, \lambda) \text{P}(i_{t-1}, \dots, i, \lambda)
$$
根据马尔可夫假设：
$$
\text{P}(i_t|i_{t-1}, \dots, i, \lambda) = \text{P}(i_t|i_{t-1}, \lambda) = a_{t, t-1}
$$
如上述步骤，最终概率如下：
$$
\text{P}(\text{I}|\lambda) = \pi (a_{1, 0}) \prod_{t=2}^T a_{t+1, t}
$$
同理：
$$
\text{P}(\text{O}|\text{I}, \lambda) = \prod_{t=1}^T b_{i, t} (o_t)
$$
$$
\text{P}(\text{O}|\lambda) = \sum_\text{I} \pi (a_{1, 0}) \prod_{t=2}^T a_{t+1, t} \prod_{t=1}^T b_{i, t} (o_t) 
$$
>
>由于此方法时间复杂度为$O(n^T)$，n为状态序列为离散值的数量，因此不适用于特别长的序列。解决办法：forward, backward https://www.bilibili.com/video/av32471608?p=3
>### 2.2.2 问题二：已知观测序列求解模型参数
>
>### 2.2.3 问题三：$argmax P(I|O, \lambda)$ 解决办法：维特比算法。

## 3. 维特比算法 Viterbi：

### 3.1 维特比算法介绍：

**viterbi 维特比算法：选择有向无环图的最短路径。**

<img src="NLP_github/viterbi_example_2.png" width="400" height="400">

https://www.zhihu.com/question/20136144

例如下图选择从S到E的最短路径，边的权重即为转换概率。

<img src="NLP_github/viterbi_example.png" width="400" height="400">

首先，选择出S到B列的最短路径，首先，分别选择S-B1，S-B2，S-B3的最短路径分别为S-A3-B1，S-A1-B2，S-A2-B3。
<img src="NLP_github/example_1.png" width="400" height="400">
基于上述提到的三条最短路径，选择S分别到C1，C2，C3的最短路径。即为：S-A3-B1-C1 S-A2-B3-C2 S-A1-B2-C3。
<img src="NLP_github/example_2.png" width="400" height="400">

最终，对比选择出最短路径，如下图所示。
<img src="NLP_github/example_3.png" width="400" height="400">

例如：命名实体标注。

<img src="NLP_github/NE.png" width="700" height="700">

横轴为状态序列，纵轴为观测序列。

* s[i, j] 得分情况，其中i为观测序列的下标，j为状态序列的下标。

* e[i, j] 为发射矩阵的元素。

* t[i, j] 为状态转移矩阵的元素。

记录选择状态节点的情况，使得score最大。

https://msd.misuland.com/pd/2884250171976188572

### 3.2 维特比算法的数据结构：

1. score matrix $ \text{V}^{|Y|×N}$ 其中，N为句子内单词的数量，Y为标签的数量。
2. Backpointer matrix。

### 3.3 算法流程：

<img src="NLP_github/viterbi_al.png" width="500" height="500">

**注意：当字典过大时，维特比算法比较耗时，使用贪心搜索可以改善此现象。**

## 4. 集束搜索 beam search：

集束搜索为维特比算法的贪心形式，每一阶段只考虑概率最大的k个标签，如下图所示。

<img src="NLP_github/beam_search.png" width="400" height="400">

<img src="NLP_github/beam_search_1.png" width="400" height="400">

## 5. 条件随机场 CRF - 无向图 - 概率判别模型

**条件随机场属于概率判别模型，解决MeMM label bias问题（MEMM倾向于选择拥有更少转移的状态）。**

条件随机场首先要定义一个特征函数集，每个特征函数都以整个句子s，当前位置i，位置i和i-1的标签为输入。然后为每一个特征函数赋予一个权重，然后针对每一个标注序列l，对所有的特征函数加权求和，必要的话，可以把求和的值转化为一个概率值。

* 特征函数：$f(s, i, l_i, l_{i-1})$ s为需要被标注的句子，i为句子中第i个词，$l_i$为第i个单词的标注，$l_{i-1}$为第i-1个单词的标注。其输出为0或1，0代表此标注不符合此特征，反之，1代表符合此特征。


* 特征函数集：多个特征对应的特征函数的集合。


* 权重$\lambda$：每一个特征函数$f_i$前有一个权重$\lambda_i$权重的大小对应此特征正确或错误的程度。例如，有一特征函数表示名词前接形容词，因此我们希望此特征函数对应的权重为正值且越大越好。相反，某一特征函数表示动词前接形容词，我们希望此权重越小越好（可以为负数）。


* $score(l|s) = \sum_{j=1}^m \sum_{i=1}^n \lambda_j f_j(s, i, l_i, l_{i-1})$ i代表句子s中的每个单词，j代表特征函数集合内的每一个特征函数。

将上述$score(l|s)$函数转换为概率表示为：

$$
P(l|s) = \frac{exp\left(score(l|s)\right)}{\sum_{l'} exp\left(score(l'|s)\right)}
$$

可以使用随机梯度下降更新换代权重：

<img src="NLP_github/CRF_train.png" width="500" height="500">

http://www.luyixian.cn/news_show_37209.aspx

条件随机场用于NER：https://zhuanlan.zhihu.com/p/111340463

## 5. HMM & CRF & LR & NB:

<img src="NLP_github/CRF.png" width="400" height="400">

逻辑回归是用于分类的对数线性模型，条件随机场是用于序列话标注的对数线性模型。

朴素贝叶斯模型和隐马尔可夫模型需要建模的是输入变量和输出变量的**联合概率分布。**

逻辑回归模型和条件随机场模型需要建模的是输入变量和输出变量的**条件概率分布。**